In [ ]:
import sys
import os
import time
import datetime as dt

import numpy as np
import scipy.ndimage
import pandas as pd
import matplotlib.pyplot as plt

from osgeo import gdal
from osgeo import osr
from osgeo.gdalconst import *
driver = gdal.GetDriverByName('GTiff')
driver.Register()


In [ ]:
def interpolate_from_storageRatio(targetStorageRatio, column):
    #print(targetStorageRatio)
    if targetStorageRatio < min(relationship_df["norm_SoilWatVol"]):
        return 0
    upperStorageRatio = min(relationship_df[relationship_df["norm_SoilWatVol"] > targetStorageRatio]["norm_SoilWatVol"])
    lowerStorageRatio = max(relationship_df[relationship_df["norm_SoilWatVol"] < targetStorageRatio]["norm_SoilWatVol"])
    #print(upperStorageRatio, lowerStorageRatio)
    upperRatio = relationship_df.loc[relationship_df["norm_SoilWatVol"] == upperStorageRatio, column].iloc[0]
    lowerRatio = relationship_df.loc[relationship_df["norm_SoilWatVol"] == lowerStorageRatio, column].iloc[0]
    if upperRatio == lowerRatio:
        return upperRatio
    
    targetRatio = lowerRatio + (targetStorageRatio-lowerStorageRatio)*(upperRatio-lowerRatio)/(upperStorageRatio-lowerStorageRatio)
    return targetRatio



def plotFlow(times, day_month_series, precip, streamFlow, streamFlow_observed, m):    
    fig,ax = plt.subplots(figsize=(14,5.5))
    plt.gcf().text(0.65, 0.8, "05-01 to 10-25-2017, hourly data\ndecay m: {}".format(m), 
                       fontsize=13, color='black')
    ax.set_title(watershed_name + ' ', fontsize=20)
    #ax.set_ylabel(r'$\frac{A_{sat}}{A_{tot}}$', fontsize=22, labelpad=22, rotation='horizontal')
    ax.set_ylabel(r'Streamflow per unit area [mm/hr]', fontsize=16, labelpad=16, rotation='vertical')
    ax.set_xlabel("Time", fontsize=16, labelpad=10, rotation='horizontal')
    spaced_yticks = [min(streamFlow),
                     min(streamFlow)+(max(streamFlow)-min(streamFlow))*0.25,
                     min(streamFlow)+(max(streamFlow)-min(streamFlow))*0.5,
                     min(streamFlow)+(max(streamFlow)-min(streamFlow))*0.75,
                     max(streamFlow)]
    ax.set_yticks(spaced_yticks)
    ax.set_yticklabels(['{0:.4f}'.format(spaced_yticks[0]),
                        '{0:.4f}'.format(spaced_yticks[1]),
                        '{0:.4f}'.format(spaced_yticks[2]),
                        '{0:.4f}'.format(spaced_yticks[3]),
                        '{0:.4f}'.format(spaced_yticks[-1])], 
                       fontsize=13)
    first_days_of_month_1 = [first for first in day_month_series if '01-' in first]
    first_days_of_month = []
    for first in first_days_of_month_1:
        if first not in first_days_of_month:
            first_days_of_month.append(first)
    print(first_days_of_month)
    first_days_of_month_indices = [31*24, 61*24, 92*24, 123*24, 153*24, 178*24, 208*64]
    print(first_days_of_month_indices[:len(first_days_of_month)])
    ax.set_xticks(first_days_of_month_indices[:len(first_days_of_month)])
    ax.set_xticklabels(first_days_of_month)
    
    ax.plot(times, streamFlow_observed, color='k', linewidth=0.7)
    ax.plot(times, streamFlow, color='r', linewidth=0.7)
    
    ax2 = ax.twinx()
    #ax2.bar(times, precip, 0.2, color='dodgerblue')
    #print(precip.head())
    ax2.invert_yaxis()
    #ax2.set_ylabel(r'Precipitation [mm]', fontsize=16, labelpad=16, rotation='vertical')
    #y2_ticks = np.linspace(0, max(precip), max(precip))
    #y2_ticklabels = [str(i) for i in y2_ticks]
    #ax2.set_yticks(-1 * y2_ticks)
    #ax2.set_yticklabels(y2_ticklabels)
    fig.show()

    
    
def plotDeficitRatio(times, day_month_series, precip, deficitRatio, m):
    deficitRatio = deficitRatio*100
    fig,ax = plt.subplots(figsize=(14, 5.5))
    plt.gcf().text(0.65, 0.8, "05-01 to 10-25-2017, hourly data\ndecay m: {}".format(m), 
                       fontsize=13, color='black')
    ax.set_title(watershed_name + ' ', fontsize=20)
    #ax.set_ylabel(r'$\frac{A_{sat}}{A_{tot}}$', fontsize=22, labelpad=22, rotation='horizontal')
    ax.set_ylabel(r'% water deficit', fontsize=16, labelpad=16, rotation='vertical')
    ax.set_xlabel("Time", fontsize=16, labelpad=10, rotation='horizontal')
    spaced_yticks = [min(deficitRatio),
                     min(deficitRatio)+(max(deficitRatio)-min(deficitRatio))*0.25,
                     min(deficitRatio)+(max(deficitRatio)-min(deficitRatio))*0.5,
                     min(deficitRatio)+(max(deficitRatio)-min(deficitRatio))*0.75,
                     max(deficitRatio)]
    ax.set_yticks(spaced_yticks)
    ax.set_yticklabels(['{0:.2f}'.format(spaced_yticks[0]),
                        '{0:.2f}'.format(spaced_yticks[1]),
                        '{0:.2f}'.format(spaced_yticks[2]),
                        '{0:.2f}'.format(spaced_yticks[3]),
                        '{0:.2f}'.format(spaced_yticks[-1])], 
                       fontsize=13)
    ax.plot(times, deficitRatio, color='r', linewidth=0.7)
    
    ax2 = ax.twinx()
    ax2.bar(times, precip, 0.2, color='dodgerblue')
    #print(precip.head())
    ax2.invert_yaxis()
    ax2.set_ylabel(r'Precipitation [mm]', fontsize=16, labelpad=16, rotation='vertical')
    #y2_ticks = np.linspace(0, max(precip), max(precip)+1)
    #y2_ticklabels = [str(i) for i in y2_ticks]
    #ax2.set_yticks(-1 * y2_ticks)
    #ax2.set_yticklabels(y2_ticklabels)
    fig.show()


    
def plotAreaRatio(times, day_month_series, precip, areaRatio, m):
    areaRatio = areaRatio*100
    fig,ax = plt.subplots(figsize=(14,5.5))
    plt.gcf().text(0.65, 0.8, "05-01 to 10-25-2017, hourly data\ndecay m: {}".format(m), 
                       fontsize=13, color='black')
    ax.set_title(watershed_name + ' ', fontsize=20)
    #ax.set_ylabel(r'$\frac{A_{sat}}{A_{tot}}$', fontsize=22, labelpad=22, rotation='horizontal')
    ax.set_ylabel(r'% saturated area', fontsize=16, labelpad=16, rotation='vertical')
    ax.set_xlabel("Time", fontsize=16, labelpad=10, rotation='horizontal')
    spaced_yticks = [min(areaRatio),
                     min(areaRatio)+(max(areaRatio)-min(areaRatio))*0.25,
                     min(areaRatio)+(max(areaRatio)-min(areaRatio))*0.5,
                     min(areaRatio)+(max(areaRatio)-min(areaRatio))*0.75,
                     max(areaRatio)]
    ax.set_yticks(spaced_yticks)
    ax.set_yticklabels(['{0:.2f}'.format(spaced_yticks[0]),
                        '{0:.2f}'.format(spaced_yticks[1]),
                        '{0:.2f}'.format(spaced_yticks[2]),
                        '{0:.2f}'.format(spaced_yticks[3]),
                        '{0:.2f}'.format(spaced_yticks[-1])], 
                       fontsize=13)
    ax.plot(times, areaRatio, color='r', linewidth=0.7)
    
    ax2 = ax.twinx()
    ax2.bar(times, precip, 0.2, color='dodgerblue')
    #print(precip.head())
    ax2.invert_yaxis()
    ax2.set_ylabel(r'Precipitation [mm]', fontsize=16, labelpad=16, rotation='vertical')
    #y2_ticks = np.linspace(0, max(precip), max(precip)+1)
    #y2_ticklabels = [str(i) for i in y2_ticks]
    #ax2.set_yticks(-1 * y2_ticks)
    #ax2.set_yticklabels(y2_ticklabels)
    fig.show()

In [ ]:
dir_path = os.path.realpath('./')
cwd = os.getcwd()

watershed_name = "Hubbard Brook #7"
watershed_foldername = "hubbard_brook"
watershed_nick = "hub7"
working_dir = (os.path.abspath(os.path.join(dir_path,'..')) + "/data/" + 
               watershed_foldername + os.path.sep +
               watershed_nick + os.path.sep)
n_days = 20000
run_number = 1
decaying_transmissivity = True
m = 0.015


dem = working_dir + watershed_nick + "_10dem.tif"
elevations_ds = gdal.Open(dem, GA_ReadOnly)
band = elevations_ds.GetRasterBand(1)
cols, rows = elevations_ds.RasterXSize, elevations_ds.RasterYSize
elevationValues = band.ReadAsArray(0, 0, cols, rows)
geotransform = elevations_ds.GetGeoTransform()
resol = np.round(geotransform[1])
watershed_cells = np.where(elevationValues>0.0, 1, 0)
A = np.sum(watershed_cells) * resol * resol
print(A)

#parameters for sand  
Ks = 20  #micrometers per second
Ks = Ks * 3600 / 1000000  #meters per hourd
psi_b = 0.1  #bubbling pressure / capillary head, for sand
fi = 0.45  #porosity
Sy = 0.24  #specific yield
b = 0.00010
n = 140

D = 2
#ET = 3/24
ETdict = {'Apr':0.003/24, 'May':0.003/24, 'Jun':0.003/24, 'Jul':0.003/24,
     'Aug':0.003/24, 'Sep':0.003/24, 'Oct':0.003/24}
maxStorage = A * D * fi



run_foldername = "{}_{}_{}".format(watershed_nick, n_days, run_number) + os.path.sep
relationship_table_filename = run_foldername+watershed_nick+"_{}_{}_noDecay_ARatio_satFront_SoilWatVol.txt".format(n_days, run_number)
if decaying_transmissivity == True:
    run_foldername = "{}_{}_{}_decay{}".format(watershed_nick, n_days, run_number, m) + os.path.sep
    relationship_table_filename = run_foldername+watershed_nick+"_{}_{}_decay{}_ARatio_satFront_SoilWatVol.txt".format(n_days, run_number, m)
relationship_df = pd.read_csv(relationship_table_filename, header=0, sep='\t', index_col=0)
relationship_df.head()

precipitation_column_names = ["timestep", "day", "hour", "precip"]
precipitation_filename = working_dir+watershed_nick+"__2014_0501_1231_precipitation.csv"
precipitation_df = pd.read_csv(precipitation_filename, header=0, index_col=0, names=precipitation_column_names)
date_time_obj = pd.to_datetime(precipitation_df.day)
day_month = date_time_obj.apply(lambda x: x.strftime("%d-%b"))
precipitation_df["day_month"] = day_month
precipitation_df = precipitation_df[precipitation_df["day"] >= "2014-05-04"]
precipitation_df = precipitation_df[precipitation_df["day"] <= "2014-10-25"]
print(precipitation_df.head(), precipitation_df.tail())
print(precipitation_df.shape)

observation_filename = working_dir+watershed_nick+"_2014_hourly_data.csv"
observation_column_names = ["timestep","day","hour","streamflow_cfs","streamflow_ms","ET_mhr"]
observation_df = pd.read_csv(observation_filename, header=0, index_col=0, 
                             usecols=[0,1,2,3,5,6], names=observation_column_names)
date_time_obj = pd.to_datetime(observation_df.day)
day_month = date_time_obj.apply(lambda x: x.strftime("%d-%b"))
observation_df["day_month"] = day_month
correct_date_format_Series = date_time_obj.apply(lambda x: x.strftime("%Y-%m-%d"))
observation_df["day"] = correct_date_format_Series
observation_df = observation_df[observation_df["day"] >= "2014-05-04"]
observation_df = observation_df[observation_df["day"] <= "2014-10-25"]
print(observation_df.head(), observation_df.tail())
print(observation_df.shape)

In [ ]:
output_Dict = {}
initialStorageRatio = 0.98
initialStorageVol = maxStorage * initialStorageRatio
initialDeficitRatio = 1 - initialStorageRatio
initialDeficitVol = maxStorage * initialDeficitRatio
initialAreaRatio = interpolate_from_storageRatio(initialStorageRatio, "AreaRatio")
print("initialAreaRatio is {}".format(initialAreaRatio))
initialAsat = initialAreaRatio * A
#initialFront = interpolate_from_storageRatio(initialStorageRatio, "Saturated_Front")

currentAsat = initialAsat
currentStorageVol = initialStorageVol
currentStorageRatio = initialStorageRatio
print("initialStorageRatio is {}".format(initialStorageRatio))
#currentFront = initialFront

timesteps = precipitation_df.index
#initial_timestep, final_timestep = min(timesteps), min(timesteps)+(24*(31*3+30*2+24))
initial_timestep = min(timesteps) #+ 24*(31+23)
final_timestep = initial_timestep + (24*(28+31*2+30*2+24))-1 #(24*(7+7)) - 1
print(initial_timestep, final_timestep)
range(min(timesteps),max(timesteps))

observed_streamFlow_mmhr = observation_df["streamflow_ms"].loc[initial_timestep:final_timestep]*3600*1000  #mm/hr
b_estimate = (observation_df["streamflow_ms"].iloc[0]*3600)/(initialStorageRatio)**n  #m/hr
#print(observed_streamFlow_mmhr, b_estimate)

In [ ]:
sumP = 0
sumET = 0
sumQrunoff = 0
sumQsubsurface = 0
sumStreamFlow = 0
sumStorageVol = initialStorageVol
finalStorageVol = 0

for timestep in range(initial_timestep, final_timestep+1):
    print("Running timestep {}".format(timestep))
    previousAsat = currentAsat
    previousStorageVol = currentStorageVol
    previousStorageRatio = currentStorageRatio
    #previousFront = currentFront
    
    date = precipitation_df['day'].loc[timestep]
    hour = precipitation_df['hour'].loc[timestep]
    day_month = precipitation_df['day_month'].loc[timestep]
    P = precipitation_df['precip'].loc[timestep]/1000
    totalP = P * A
    sumP += totalP
    Qrunoff = P * previousAsat
    Qsubsurface = b_estimate * A * previousStorageRatio**n
    ET = observation_df['ET_mhr'].loc[timestep]*A  #m/hr
    #ET = ETdict[day_month[-3:]]*A
    sumET += ET
    deltaStorageVol = totalP - ET - Qrunoff - Qsubsurface
    currentStorageVol = previousStorageVol + deltaStorageVol
    if currentStorageVol < 0:
        #print("CAZZATONAAAAAAAAAAAAAAA\n\n\n\n\n\n\n\n\n\n\n\n")
        currentStorageVol = 0
    if currentStorageVol > maxStorage:
        print("\nSATURATION EXCESS!!\n")
        Qrunoff += currentStorageVol - maxStorage
        Qsubsurface = b_estimate * A * 1**n
        deltaStorageVol = totalP - ET - Qrunoff - Qsubsurface
        currentStorageVol = previousStorageVol + deltaStorageVol
    norm_Qrunoff = Qrunoff / A
    norm_Qsubsurface = Qsubsurface / A
    streamFlow = Qrunoff + Qsubsurface
    norm_streamflow = norm_Qrunoff + norm_Qsubsurface
    sumQrunoff += Qrunoff
    sumQsubsurface += Qsubsurface
    #streamFlow = Qrunoff + Qsubsurface
    sumStreamFlow += streamFlow
    
    currentStorageRatio = currentStorageVol/maxStorage
    currentAreaRatio = interpolate_from_storageRatio(currentStorageRatio, "AreaRatio")
    currentAsat = currentAreaRatio * A
    #currentFront = interpolate_from_storageRatio(currentStorageRatio, "Saturated_Front")
    output_Dict[timestep] = [date, hour, day_month,
                             P*1000, Qrunoff, Qsubsurface,
                             currentStorageRatio, currentAreaRatio,
                             streamFlow, norm_streamflow*1000]
    finalStorageVol = currentStorageVol

print()
print(sumP, sumET, sumQrunoff, sumQsubsurface, sumStreamFlow, sumStorageVol)
print("Final water balance is {} m^3".format(sumP - sumET - sumStreamFlow))
print("{} - {} = {}".format(finalStorageVol, initialStorageVol, finalStorageVol-initialStorageVol))

output_df = pd.DataFrame.from_dict(output_Dict,
                                   orient = 'index',
                                   columns = ["date", "hour", "day_month",
                                              "P", "Qse","Qgw", 
                                              "storageRatio", "areaRatio",
                                              "model_m3hr", "model_mmhr"])

output_df["observ_mmhr"] = observed_streamFlow_mmhr

model_output_name = "{}{}_{}_{}_noDecay_modelOutput.csv".format(run_foldername, watershed_nick, 
                                                                               n_days, run_number)
if decaying_transmissivity == True:
    model_output_name = "{}{}_{}_{}_decay{}_modelOutput.csv".format(run_foldername, watershed_nick, 
                                                                             n_days, run_number, m)
print(model_output_name)   
output_df.to_csv(model_output_name, sep=',')

timesteps = output_df.index
day_month_series = output_df["day_month"]
hour_series = output_df["hour"]
date_series = output_df["date"]
precip = output_df["P"]
Qse = output_df["Qse"]
Qgw = output_df["Qgw"]
streamFlow = output_df["model_m3hr"]
norm_streamFlow = output_df["model_mmhr"]
storageRatio = output_df["storageRatio"]
areaRatio = output_df["areaRatio"]
#print(observed_streamFlow)

In [ ]:
print(observed_streamFlow_mmhr.head())
plotFlow(timesteps[(28+23)*24:(28+30+7)*24], day_month_series[(28+23)*24:(28+30+7)*24], 
         precip[(28+23)*24:(28+30+7)*24],
         norm_streamFlow[(28+23)*24:(28+30+7)*24],
         observed_streamFlow_mmhr[(28+23)*24:(28+30+7)*24], m)
plotDeficitRatio(timesteps[(28+23)*24:(28+30+7)*24], day_month_series[(28+23)*24:(28+30+7)*24], 
                 precip[(28+23)*24:(28+30+7)*24], 
                 1-storageRatio[(28+23)*24:(28+30+7)*24], m)
plotAreaRatio(timesteps[(28+23)*24:(28+30+7)*24], day_month_series[(28+23)*24:(28+30+7)*24],
              precip[(28+23)*24:(28+30+7)*24],
              areaRatio[(28+23)*24:(28+30+7)*24], m)

In [ ]:
btry = observed_streamFlow.iloc[1]/3600/(0.89)**0.6
print(observed_streamFlow.iloc[1])
print(btry)

In [ ]:
print(resol)